In [66]:
import pandas as pd
import re
from bs4 import BeautifulSoup as BS
import requests
from tqdm.notebook import tqdm
from time import sleep
from random import random

In [67]:
def get_queue(cookies: dict = None):
    """
    Получаем ДатаФрейм с сылками на категории и подкатегории
    Для дальнейшего парсинга.
    """
    
    url = 'https://pack24.ru/'
    response = requests.get(url, cookies=cookies, timeout=10)
    if response.status_code == 200:
        # return response.text
        soup = BS(response.text, 'html.parser')

        tk1_name = []
        tk1_link = []
        tk1_id = []

        elem : BS
        for elem in list(soup.find('header').find_next_sibling('div').find_all('li'))[:]:
            tk1_link.append(elem.find('a').get('href'))
            tk1_name.append(elem.find('a').get_text(strip=True))
            tk1_id.append(elem.find('a').get('data-id'))

        df1 = pd.DataFrame({
            'tk1_id': tk1_id,
            'tk1_name': tk1_name,
            'tk1_link': tk1_link,
        })

        tk2_name = []
        tk2_link = []
        tk1_id = []

        elem : BS
        for elem in list(soup.find('div', class_="catalog-submenu").find_all('ul'))[:]:
            item: BS
            for item in list(elem.find_all('li'))[:]:
                tk1_id.append(elem.get('id')[6:])
                tk2_link.append(item.find('a').get('href'))
                tk2_name.append(item.find('a').get_text(strip=True))

        df2 = pd.DataFrame({
            'tk1_id': tk1_id,
            'tk2_name': tk2_name,
            'tk2_link': tk2_link,
        })

        return df1.merge(right=df2, how='right').convert_dtypes().astype({'tk1_id': 'Int16'})
    return Exception('status_code not 200')

In [68]:
def worker_01(ss: BS) -> pd.DataFrame:
    """
    Парсинг sku с первого типа страниц
    """
    sku_name = []
    sku_href = []
    sku_desc = []
    price_old = []
    price_old_sale = []
    price = []
    price_opt = []
    nds = []

    item: BS
    desc : BS
    for item in ss.find('div', class_='catalog').find_all('div', class_="item__body")[:]:
        sku_name.append(item.find('a', class_='item__name').get_text(strip=True))
        sku_href.append(item.find('a', class_='item__name').get('href'))
        if li := item.find('ul', class_='item__desc').find_all('li'):
        # Парсинг свойств.
            item_desc = {}
            for desc in li:
                try:
                    item_desc[desc.find('span').get_text(strip=True)] = re.sub(r'\n*\s+', ' ', desc.find('span').next_sibling.get_text(strip=True))
                except AttributeError:
                    continue
            sku_desc.append(item_desc)
        
        # price_old
        price_old_soup = item.find('div', class_='item__info').find('div', class_="item__price--with-discount")
        try:
            price_old.append(re.sub(r'\s', '',
                                re.findall(r'(\d{,3}\s?\d+.\d+)', price_old_soup.find('div').get_text())[0]))
        except AttributeError:
            price_old.append(None)
        try:
            price_old_sale.append(price_old_soup.find('div').find_next_sibling('div').get_text(strip=True))
        except AttributeError:
            price_old_sale.append(None)
        # price
        price.append(re.sub(r'\s', '',
                        re.findall(r'(\d{,3}\s?\d+.\d+)', item.find('div', class_='item__price').find('span').previous_element.get_text(strip=True))[0]))
        nds.append(re.sub(r'\s', '',
                        re.findall(r'(\d{,3}\s?\d+.\d+)', item.find('div', class_='flex').find('p').find_next_sibling('p').get_text(strip=True))[0]))
        price_opt.append(re.sub(r'\s', '',
                            re.findall(r'(\d{,3}\s?\d+.\d+)', item.find('div', class_='item__info').find('div', class_="inline-block").get_text(strip=True))[0]))

    df = pd.DataFrame({
                        'sku_name': sku_name,
                        'sku_href': sku_href,
                        'sku_desc': sku_desc,
                        'price_old': price_old,
                        'price_old_sale': price_old_sale,
                        'price': price,
                        'price_opt': price_opt,
                        'nds': nds,
                    })
    return df
        

In [69]:
def worket_02(ss: BS) -> pd.DataFrame:
    """
    Парсинг sku с второго типа страниц
    """
    sku_name = []
    sku_href = []
    sku_desc = []
    price_old = []
    price_old_sale = []
    price = []
    price_opt = []
    nds = []

    item: BS
    desc : BS

    for item in ss.find('div', class_='category js_category').find_all('li', class_='category-item')[:]:
        sku_name.append(item.find('meta').get('content').strip())
        sku_href.append(item.find('a').get('href'))
        if li := item.find('ul').find_all('li'):
        # Парсинг свойств.
            item_desc = {}
            for index, desc in enumerate(li):
                item_desc[f'Описание {index+1}'] = desc.get_text(strip=True)
            sku_desc.append(item_desc)

        # # price_old
        price_old.append(None)
        price_old_sale.append(None)
        # price
        price.append(re.sub(r'\s', '',
                        (re.findall(r'(\d{,3}\s?\d+.\d+)', item.find('span', class_="category-item__pricy").get_text(strip=True))[0].replace(',','.'))))
        nds.append(None)
        price_opt.append(None)
    
    df = pd.DataFrame({
                        'sku_name': sku_name,
                        'sku_href': sku_href,
                        'sku_desc': sku_desc,
                        'price_old': price_old,
                        'price_old_sale': price_old_sale,
                        'price': price,
                        'price_opt': price_opt,
                        'nds': nds,
                    })
    return df

In [78]:
def get_sku(queue_df: pd.DataFrame, cookies: dict = None):
    """
    Запуск парсера по очереди, в зависимости от типа страницы.
    Есть пара страниц, без sku.
    
    Так же на сайте присустсвует защита по ip.
    Для того что бы обойти ее, необходимо перейти по появишейся ссылке,
    после в запросе к документу странице найти среди cookies: ihead_captcha_#############
    Его значение передать в параметр token
    """
    queue_list = list(map(lambda x: dict(x._asdict()), queue_df.itertuples(index=False)))

    df_list = []

    for row in tqdm(queue_list, desc='Downloads'):
        for _ in range(3):
            try:
                sleep(random() + 0.1)
                response = requests.get(row.get('tk2_link'), cookies=cookies, timeout=10)
            except Exception:
                continue
            else:
                break
        else:
            print(('Error', row.get('tk2_link')))
            continue
        
        soup = BS(response.text, 'html.parser')

        try:
            df_tmp = worker_01(soup).assign(tk2_name = row.get('tk2_name'))
        except Exception:
            try:
                df_tmp = worket_02(soup).assign(tk2_name = row.get('tk2_name'))
            except AttributeError:
                print(row.get('tk2_link'))
                df_tmp = pd.DataFrame()
        if df_tmp.shape[0]:
            df_list.append(df_tmp)
    return df_list

In [71]:
cookies = None
# cookies = {'ihead_captcha_hR5qU8sK4yG6':'3dcTjjg0Nd%21%5ENN%21R'}

In [72]:
"""ПОЛУЧЕНИЕ ОЧЕРЕДИ"""
queue = get_queue(cookies)
if isinstance(queue, Exception):
    print('status_code not 200')

In [79]:
"""ПОЛУЧЕНИЕ ДАННЫХ"""
# df_list = get_sku(queue, '%2F%40r2-%40%40%21BuYF%2Ai0b')
df_list = get_sku(queue, cookies)
df = (pd.concat(df_list, ignore_index=True).convert_dtypes()
    .astype({
        'price_old': 'Float64',
        'price': 'Float64',
        'price_opt': 'Float64',
        'nds': 'Float64',
    })
    )


Downloads:   0%|          | 0/106 [00:00<?, ?it/s]

https://pack24.ru/upakovochnaya-bumaga/bumaga-ofisnaya
https://pack24.ru/termopakety-termosumki/termosumki


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1718 entries, 0 to 1717
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sku_name        1718 non-null   string 
 1   sku_href        1718 non-null   string 
 2   sku_desc        1718 non-null   object 
 3   price_old       143 non-null    Float64
 4   price_old_sale  143 non-null    string 
 5   price           1718 non-null   Float64
 6   price_opt       1713 non-null   Float64
 7   nds             1713 non-null   Float64
 8   tk2_name        1718 non-null   string 
dtypes: Float64(4), object(1), string(4)
memory usage: 127.6+ KB


In [75]:
df.head()

,sku_name,sku_href,sku_desc,price_old,price_old_sale,price,price_opt,nds,tk2_name
0,Гофрокороб 80*80*80 мм,https://pack24.ru/kartonnye-korobki/chetyrehkl...,"{'Размер': '80*80*80 мм (Д*Ш*В), объем 0,512 л...",<NA>,<NA>,10.55,9.0,1.76,Четырехклапанные гофрокороба
1,Гофрокороб 100*100*100 мм,https://pack24.ru/kartonnye-korobki/chetyrehkl...,"{'Размер': '100*100*100 мм (Д*Ш*В), объем - 1 ...",<NA>,<NA>,10.75,9.15,1.79,Четырехклапанные гофрокороба
2,Гофрокороб 100*100*200 мм,https://pack24.ru/kartonnye-korobki/chetyrehkl...,"{'Размер': '100*100*200 мм (Д*Ш*В), объем - 2 ...",<NA>,<NA>,15.7,13.35,2.62,Четырехклапанные гофрокороба
3,Гофрокороб 120*120*120 мм,https://pack24.ru/kartonnye-korobki/chetyrehkl...,"{'Размер': '120*120*120 мм (Д*Ш*В), объем 1,73...",<NA>,<NA>,11.3,9.65,1.88,Четырехклапанные гофрокороба
4,Гофрокороб 135*135*135 мм,https://pack24.ru/kartonnye-korobki/chetyrehkl...,"{'Размер': '135*135*135 мм (Д*Ш*В), объем 2,46...",<NA>,<NA>,12.3,10.45,2.05,Четырехклапанные гофрокороба


In [76]:
df.to_excel('result.xlsx', index=True)
df.to_parquet('result.parquet', engine='pyarrow', compression='zstd')